<a href="https://colab.research.google.com/github/ultimatelala/Git_doc/blob/master/tatanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train=pd.read_csv(r'train.csv')
test=pd.read_csv(r'test.csv')
print('train_data',train.shape,'test_data',test.shape)
full=train.append(test,ignore_index=True)
full.describe()
full.info()

**处理异常值**

In [ ]:
#处理异常值（加入平均数（定量），加入最常见数组（定性））
full['Age'].fillna(full['Age'].mean(),inplace=True)
full['Fare'].fillna(full['Fare'].mean(),inplace=True)
full['Embarked'].value_counts()
full['Embarked'].fillna('S',inplace=True)
#缺失值过多，用‘U’替代
full['Cabin'].head(10)
full['Cabin'].fillna('U',inplace=True)

**特征工程**

性别处理

In [ ]:
full['Sex']=full['Sex'].map({'male':1,'female':0})

港口one—hot编码

In [ ]:
full['Embarked'].head()
embark=pd.get_dummies(full['Embarked'],prefix='port')
full=pd.concat([full,embark],axis=1)
full.drop('Embarked',axis=1,inplace=True)

船仓one—hot编码

In [ ]:
pclass=pd.get_dummies(full['Pclass'],prefix='Pclass')
full=pd.concat([full,pclass],axis=1)
full.drop('Pclass', axis=1,inplace=True)

姓名处理

In [ ]:
full['Name'].head()
def title(name):
    str1=name.split(',')[1]
    str2=str1.split('.')[0].strip()
    return str2
full['Name']=full['Name'].map(title)
full['Name'].value_counts()
title_mapDict ={'Capt':'officer','Col':'Officer','Major':'Officer','Jonkheer':'Royalty','Don':'Royalty','Sir':'Royalty','Dr':'Officer','Rev':'Officer','the Countess':'Royalty','Dona':'Royalty','Mme':'Mrs','Mlle':'Miss','Ms':'Mrs','Mr':'Mr','Mrs':'Mrs','Miss':'Miss','Master':'Master','Lady':'Royalty'} 
full['Name'] = full['Name'].map(title_mapDict)
name=pd.get_dummies(full['Name'])
full=pd.concat([full,name],axis=1)
full.drop('Name',axis=1,inplace=True)

客舱one—hot编码

In [ ]:
full['Cabin'].value_counts()
full['Cabin']=full['Cabin'].map(lambda x:x[0])
Cabin=pd.get_dummies(full['Cabin'],prefix='Cabin')
full=pd.concat([full,Cabin],axis=1)
full.drop('Cabin',axis=1,inplace=True)

家庭成员处理

In [ ]:
family=pd.DataFrame()
family['number']=full['Parch']+full['SibSp']+1
family['single']=family['number'].map(lambda x:1 if x==1 else 0)
family['small']=family['number'].map(lambda x:1 if 2<=x<=4 else 0)
family['large']=family['number'].map(lambda x:1 if x>4 else 0)
family.head(15)

查看各元素和生还率的相关值


In [ ]:
full=pd.concat([full,family],axis=1)
full.corr()['Survived'].sort_values(ascending=False)
full_x=pd.concat([name,pclass,family,full['Fare'],full['Sex'],Cabin],axis=1)
sourcerow=891
source_x=full_x.loc[0:sourcerow-1,:]
source_y=full.loc[:sourcerow-1,'Survived']
pred_x=full_x.loc[sourcerow:,:]

**开始机器训练**

In [ ]:
from sklearn.cross_validation import train_test_split
train_x,test_x,train_y,test_y=train_test_split(source_x,source_y,train_size=0.8)
print ('原始数据集特征：',source_x.shape, 
       '训练数据集特征：',train_x.shape ,
      '测试数据集特征：',test_x.shape)

print ('原始数据集标签：',source_y.shape, 
       '训练数据集标签：',train_y.shape ,
      '测试数据集标签：',test_y.shape)
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit( train_x, train_y )
model.score(test_x,test_y)

**上传正确格式CSV文件**

In [ ]:
pred_y=model.predict(pred_x)
pred_y=pred_y.astype(int)
passenger_id=full.loc[sourcerow:,'PassengerId']
pred=pd.DataFrame({'PassengerId':passenger_id,'Survived':pred_y})
pred.reset_index(inplace=True,drop=True)
pred.head()
pred.to_csv(r'D:\DA\titanic\final.csv',index=False)